In [86]:
#KNN
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


np.random.seed(357823)#Fijar semilla para reproducibilidad


In [87]:
#Leer csv
nodes = pd.read_csv('../Tablas/TablaAtributos.csv')
nodes.head(10)

,id,name,ml_target,Closeness_Centrality,Betweenness_Centrality,Degree_Centrality,Clustering_Coefficient,Triangles,Squares,K_Core,Comunidad,asyn_lpa_community
0,0,Eiryyy,0.0,0.350119,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.402395,4.265277e-06,0.000740,0.178571,0.000062,0.100888,0.151515,0.002227,0.0
2,2,JpMCarrilho,1.0,0.315639,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.359848,1.972237e-04,0.000423,0.000000,0.000000,0.026745,0.090909,0.004454,0.0
4,4,sunilangadi2,1.0,0.266483,2.275709e-08,0.000106,0.000000,0.000000,0.000000,0.030303,0.011136,0.0
5,5,j6montoya,0.0,0.529355,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,6,sfate,0.0,0.604897,7.785208e-06,0.000529,0.333333,0.000062,0.054200,0.151515,0.000000,0.0
7,7,amituuush,0.0,0.468539,2.286921e-06,0.000740,0.321429,0.000112,0.075637,0.212121,0.000000,0.0
8,8,mauroherlein,0.0,0.550634,6.618355e-07,0.000740,0.750000,0.000262,0.080455,0.212121,0.002227,0.0
9,9,ParadoxZero,0.0,0.529822,1.006049e-04,0.000634,0.238095,0.000062,0.002749,0.121212,0.004454,0.0


In [88]:
ac=['Closeness_Centrality','Betweenness_Centrality','Degree_Centrality','Clustering_Coefficient','Triangles','Squares','K_Core', 'Comunidad','asyn_lpa_community']
ad=['name']
atributtes = nodes.loc[:, ac + ad + ['id']]

#Elegimos el atributo a predecir
y = nodes['ml_target']


In [89]:
#KNN no trabaja con cadenas de texto por lo que se debe de transformar a valores numericos
codificador_ad = OrdinalEncoder()
codificador_ad.fit(atributtes[ad])

OrdinalEncoder()

In [90]:
#Transformamos los datos
atributtes[ad] = codificador_ad.transform(atributtes[ad])

#Normalizamos la columna nombre 
scaler = MinMaxScaler(
    feature_range=(0, 1)
)
atributtes[ad] = scaler.fit_transform(atributtes[['name']])

In [91]:
#Hiperparametros
#Para entrenar el KNN elejimos los hiperparametros , con la busqueda de rejilla 
#Se elige el mejor hiperparametro para el modelo
tub_kNN=Pipeline([
    ('kNN', KNeighborsClassifier())
])
parámetros = {
    'kNN__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25],
    'kNN__metric': [ 'euclidean','manhattan']
}
#Busqueda de REjilla con validacion cruzada
rejilla = GridSearchCV( tub_kNN, parámetros, scoring ='recall', cv=10)
rejilla.fit(atributtes, y)


GridSearchCV(cv=10, estimator=Pipeline(steps=[('kNN', KNeighborsClassifier())]),
             param_grid={'kNN__metric': ['euclidean', 'manhattan'],
                         'kNN__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19,
                                              21, 23, 25]},
             scoring='recall')

In [92]:
#Imprimir los mejores hiperparametros encontrados 
rejilla.best_score_
rejilla.best_params_
print(f"Mejores hiperparámetros: {rejilla.best_params_}")
print(f"Mejor puntaje de recall: {rejilla.best_score_}")

Mejores hiperparámetros: {'kNN__metric': 'manhattan', 'kNN__n_neighbors': 1}
Mejor puntaje de recall: 0.5438398357289527


In [93]:
#Validar
#Para ver si los resultados son correctos se realiza una validacion por retención 
#Se divide el conjunto de datos en dos partes, una para entrenar y otra para probar
X_train, X_test, y_train, y_test = train_test_split(atributtes, y, test_size=0.2,stratify=y, random_state=42)


In [97]:
#Evaluar diferentes configuraciones del modelo Knn
configuraciones = [
    (1, 'manhattan'),
    (3, 'manhattan'),
    (5, 'manhattan'),
    (1, 'euclidean'),
    (3, 'euclidean')
]

In [98]:
for n_neighbors, metric in configuraciones:
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric)
    knn.fit(X_train, y_train)
    predict = knn.predict(X_test)
    recall = recall_score(y_test, predict, average='weighted')
    print(f"Configuración: n_neighbors={n_neighbors}, metric={metric}, Recall={recall:.4f}")
    confusionM = confusion_matrix(y_test, predict)
    tabla_confusion = pd.DataFrame(confusionM, index=['VN', 'VP'], columns=['PN', 'PP'])
    print(tabla_confusion)

Configuración: n_neighbors=1, metric=manhattan, Recall=0.6259
      PN    PP
VN  4151  1441
VP  1380   568
Configuración: n_neighbors=3, metric=manhattan, Recall=0.6728
      PN   PP
VN  4698  894
VP  1573  375
Configuración: n_neighbors=5, metric=manhattan, Recall=0.6907
      PN   PP
VN  4955  637
VP  1695  253
Configuración: n_neighbors=1, metric=euclidean, Recall=0.6275
      PN    PP
VN  4172  1420
VP  1389   559
Configuración: n_neighbors=3, metric=euclidean, Recall=0.6719
      PN   PP
VN  4701  891
VP  1583  365


In [96]:
# Análisis y conclusiones
print("### Conclusiones ###\n")
print("Al evaluar los resultados, podemos observar que el modelo k-NN muestra un mejor rendimiento con `metric='manhattan'` en comparación con `metric='euclidean'`.")
print("Además, a medida que aumenta el número de vecinos (`n_neighbors`), generalmente vemos una ligera mejora en el `recall`, aunque con un costo potencial de aumentar los falsos positivos (FP).\n")

print("1. **Mejor Configuración:** La configuración con `n_neighbors=5` y `metric='manhattan'` alcanza el mayor `recall` de 0.6907, lo que indica que este modelo puede identificar correctamente el 69.07% de los verdaderos positivos en el conjunto de prueba.\n")

print("2. **Comparación de Métricas:** La métrica `manhattan` parece adaptarse mejor a la estructura de los datos en este contexto específico, mostrando consistentemente mejores resultados que `euclidean`.\n")

print("3. **Matriz de Confusión:** Observamos que las matrices de confusión revelan un desafío significativo en la predicción de la clase positiva (`VP`), con un número relativamente alto de falsos positivos en todas las configuraciones probadas.\n")

print("En resumen, estos resultados destacan la importancia de elegir cuidadosamente los hiperparámetros y la métrica de distancia en el modelo k-NN. Aunque hemos logrado un `recall` prometedor, es crucial abordar los falsos positivos y explorar estrategias adicionales para mejorar la precisión del modelo en la predicción de desarrolladores de IA en un conjunto de datos desequilibrado.")

### Conclusiones ###

Al evaluar los resultados, podemos observar que el modelo k-NN muestra un mejor rendimiento con `metric='manhattan'` en comparación con `metric='euclidean'`.
Además, a medida que aumenta el número de vecinos (`n_neighbors`), generalmente vemos una ligera mejora en el `recall`, aunque con un costo potencial de aumentar los falsos positivos (FP).

1. **Mejor Configuración:** La configuración con `n_neighbors=5` y `metric='manhattan'` alcanza el mayor `recall` de 0.6907, lo que indica que este modelo puede identificar correctamente el 69.07% de los verdaderos positivos en el conjunto de prueba.

2. **Comparación de Métricas:** La métrica `manhattan` parece adaptarse mejor a la estructura de los datos en este contexto específico, mostrando consistentemente mejores resultados que `euclidean`.

3. **Matriz de Confusión:** Observamos que las matrices de confusión revelan un desafío significativo en la predicción de la clase positiva (`VP`), con un número relativamente al